## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton:

http://www.genome.jp/kegg/rest/keggapi.html

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import math
import seaborn as sns
import urllib2
import random as rd
import Bio
import re
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from scipy.stats import hypergeom

In [2]:
# preparation for pathway drawing
from Bio import SeqIO
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from Bio.Graphics.ColorSpiral import ColorSpiral

from IPython.display import Image, HTML

import random

# A bit of code that will help us display the PDF output
def PDF(filename):
    return HTML('<iframe src=%s width=900 height=500></iframe>' % filename)

# A bit of helper code to shorten long text
def head(text, lines=10):
    """ Print the first lines lines of the passed text.
    """
    print '\n'.join(text.split('\n')[:lines] + ['[...]'])



In [3]:
% matplotlib inline

In [4]:
!mkdir -p DayY_InOutput


### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them in a suitable Python data structure

In [5]:
# get all mus musculus pathways from kegg
pw = kegg_list("pathway","mmu").read()

In [6]:
mouse_pathway_kegg = pd.DataFrame([x.replace(":","\t",1).split("\t") for x in pw.split("\n")],
                                columns=["Type","ID","Description"])

In [7]:
# set index
mouse_pathway_kegg.set_index("ID",inplace=True,drop=False)

In [8]:
# extract genes from pathways
all_genes = {}
for pathway in mouse_pathway_kegg.ID[:-1]:
    #print pathway
    pw = kegg_get(pathway).read()
    GENES = []
    Gene = False
    for line in pw.split("\n"):
        if line.startswith("GENE"):
            Gene = True
        if line.startswith("COMPOUND"):
            Gene = False
        if Gene:
            GENES.append(line)
    all_genes[pathway] = ([re.split(r'\s{1,}', g)[2].replace(";","") 
                      for g in GENES if len(re.split(r'\s{1,}', g))>2])

In [9]:
# merge genes into one data frame with pathway as ID
all_genes_joined = {}
for key in all_genes.keys():
    all_genes_joined[key] = ",\t ".join(all_genes[key])
all_genes_df = pd.DataFrame.from_dict(all_genes_joined,orient="index")
all_genes_df.columns = ["Genes"]
# sort data frame
all_genes_df.sort_index(inplace=True)

In [10]:
# append genes to kegg pathway data frame
mouse_pathway_kegg_genes = pd.concat([mouse_pathway_kegg,all_genes_df],axis=1,join="outer")
mouse_pathway_kegg_genes.drop("Type",axis=1,inplace=True)
mouse_pathway_kegg_genes.drop(mouse_pathway_kegg_genes.ix[0],inplace=True)

In [11]:
mouse_pathway_kegg_genes.head()

,ID,Description,Genes
mmu00010,mmu00010,Glycolysis / Gluconeogenesis - Mus musculus (m...,"Hk2,\t Hk3,\t Hk1,\t Hkdc1,\t Gck,\t Gpi1,\t P..."
mmu00020,mmu00020,Citrate cycle (TCA cycle) - Mus musculus (mouse),"Cs,\t Csl,\t Acly,\t Aco2,\t Aco1,\t Idh1,\t I..."
mmu00030,mmu00030,Pentose phosphate pathway - Mus musculus (mouse),"Gpi1,\t G6pd2,\t G6pdx,\t Pgls,\t H6pd,\t Pgd,..."
mmu00040,mmu00040,Pentose and glucuronate interconversions - Mus...,"Gusb,\t Kl,\t Ugt2b5,\t Ugt1a2,\t Ugt1a6a,\t U..."
mmu00051,mmu00051,Fructose and mannose metabolism - Mus musculus...,"Mpi,\t Pmm2,\t Pmm1,\t Gmppb,\t Gmppa,\t Gmds,..."


### Subtask 1.2: Save the KEGG gene sets as a gmt file after you made sure they have the proper gene ids with respect to your DE analysis

hints: 

http://biopython.org/wiki/Annotate_Entrez_Gene_IDs

http://www.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats

In [12]:
mouse_pathway_kegg_genes.to_csv("DayY_InOutput/mouse_pathway.gmt")

## Task 2: Gene Set Enrichment

### Subtask 2.1: Read in the csv file you produced during the Differential Expression module, extract a gene list (as a python list of gene symbols) from your favorite multiple correction column (and store it in a variable)

In [13]:
DE = pd.read_csv("Day4_InOutput/multiple_comparison_fc.csv",sep="\t",index_col=0)

In [14]:
diff_reg_genes = DE["simes-hochberg"]

### Subtask 2.2: Perform gene set enrichment (Fisher's exact test or an hypergeometric test will do for our purposes) with the KEGG gene sets you extracted in Task 1 (you may want to store the results in a pandas dataframe and write them to csv)

hint:

https://genetrail2.bioinf.uni-sb.de/help?topic=set_level_statistics

In [15]:
def hypergeom_test(ct):
    """
        Hypergeometric test for given crosstable ct.
        Returns p-value
    """
    k = ct.Pathway.DE
    l = ct.Pathway.All
    m = ct.All.All
    n = ct.All.DE
    kp = (n*l)/m
    if kp >= k:
        p = hypergeom.cdf(k,m,l,n)
    else:
        p = hypergeom.sf(k-1,m,l,n)
    return p

In [16]:
def apply_hypergeom_test():
    """
        Applies the hypergeometric test scipy.stats.hypergeom 
        to all pathways in the dataframe "mouse_pathway_kegg_genes" 
        and the diff_reg_genes as defined before. 
        Return: Data Frame with p values (uncorrected) for each pathway.
    """
    # extract set of genes from pathway gene list
    pathway_gene_set = set()
    for x in mouse_pathway_kegg_genes.Genes:
        for y in x.split(",\t"):
            pathway_gene_set.add(y.strip());
    # set of genes that are represented in pathway_genes and diff_reg_genes
    gene_set = list(set(diff_reg_genes.index).intersection(pathway_gene_set))

    #prepare data frame
    crossdf = pd.DataFrame([gene_set,[False]*len(gene_set),[False]*len(gene_set)]).T
    crossdf.columns=["ID","DE","Pathway"]
    crossdf.set_index("ID",inplace=True)
    
    # apply hypergeom to all pathways
    pvals = {}
    for pathway in mouse_pathway_kegg_genes.index:
        crossdf.DE.loc[set([str(c) for c in diff_reg_genes.loc[diff_reg_genes.values<0.05].
                            index.values]).intersection(gene_set)] = True
        crossdf.Pathway.loc[
            set([str(x.strip()) for x in mouse_pathway_kegg_genes.Genes[pathway].split(",\t")]).intersection(gene_set)
                ] = True
        # calculate crosstable
        crosstable = pd.crosstab(crossdf.DE.replace([False,True],["Not DE","DE"]),
                    crossdf.Pathway.replace([False,True],["Not Pathway","Pathway"]),margins=True)
        pvals[pathway] = hypergeom_test(crosstable)
    # convert to pd.DataFrame
    pvals = pd.DataFrame.from_dict(pvals,orient="index")
    pvals.columns=(["P-Value_hypergeom"])
    return pvals

### Subtask 2.3: Extract a list of significantly (at 0.05 significance) enriched KEGG pathways

In [17]:
pvals_hg = apply_hypergeom_test()

In [18]:
enriched_kegg_pathways = pvals_hg[pvals_hg["P-Value_hypergeom"]<0.01/len(pvals_hg)]
enriched_kegg_pathways

,P-Value_hypergeom
mmu04622,1.609165e-10
mmu04022,4.699378e-07
mmu04350,9.623347e-11
mmu03022,4.107536e-07
mmu03020,6.698924e-07
mmu04140,9.507226e-08
mmu04215,4.902552e-11
mmu04640,1.938185e-11
mmu04370,3.730112e-09
mmu04392,9.059903e-09


## Task 3: KEGG map visualization

#### hint:

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

#### remark:

In real life you may want to use the R-based tool pathview: https://bioconductor.org/packages/release/bioc/html/pathview.html (if you insist you can also try to use r2py for using pathview from Python during the practical)

For Python (in addition to the Biopyhton module) https://github.com/idekerlab/py2cytoscape in combination with https://github.com/idekerlab/KEGGscape may be another alternative (in the future)

Generally speaking, it is always a good idea to pay attention also to other pathway databases like Reactome or WikiPathways ...

### Subtask 3.1: Pick some significantly enriched KEGG pathways of your choice from 2.3 and visualize them

In [19]:
def get_gene_name(gene_ids):
    """
        Return gene name from given gene id.
    """
    gene_ids = gene_ids.split(" ")
    names = []
    for id in gene_ids:
        try:
            gene_name = (kegg_get(id).read())
        except:
            gene_name = "unknown"
        [names.append(re.split(r'\s{1,}',line)[1].replace(",",""))
                 for line in gene_name.split("\n") if line.startswith("NAME")]
    return names

In [20]:
def draw_kegg_map(map_id, sig_col=None,unsig_col=None,color_list=None,color_dict=None,):
    """ 
        Render a local PDF of a KEGG map with the passed map ID
        and color pathways with differentially expressed genes if 
        sig_col and unsig_col are set. 
    """
    if unsig_col != None:
        for element in pathway.genes:
            if sig_col != None:
                for name in get_gene_name(element.name):
                    if name in diff_reg_genes.index:
                        for graphic in element.graphics:
                            graphic.bgcolor = sig_col
                    else:                
                        for graphic in element.graphics:
                            graphic.bgcolor = unsig_col
                break;
                
    canvas = KGMLCanvas(pathway, import_imagemap=True)
    img_filename = "DayY_InOutput/%s.pdf" % map_id
    canvas.draw(img_filename)
    return PDF(img_filename)

In [21]:
draw_kegg_map("mmu04152")


AttributeError: 'str' object has no attribute 'image'

In [ ]:
draw_kegg_map("mmu04723")

In [ ]:
draw_kegg_map("mmu04151")

### Subtask 3.2: Define a a suitable binary color scheme respresenting the fact whether a gene is significantly expressed or not

hint: 

http://www.rapidtables.com/web/color/RGB_Color.htm

In [ ]:
# Dark blue for significant genes
col_sig = '#0000FF'
# light red for unsignificant genes
col_unsig = '#FF99CC'

### Subtask 3.3: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.2 ( you may need to define a suitable mapping from single genes to what is actually shown in the pathway map...)

In [ ]:
draw_kegg_map("mmu04152",col_sig,col_unsig)

In [ ]:
draw_kegg_map("mmu04723",col_sig,col_unsig)

### Subtask 3.4: Define a suitable continuous color range representing the log2 fold changes of the all the genes in your data

hint:

http://bsou.io/posts/color-gradients-with-python

In [ ]:
def hex_to_RGB(hex):
  ''' "#FFFFFF" -> [255,255,255] '''
  # Pass 16 to the integer function for change of base
  return [int(hex[i:i+2], 16) for i in range(1,6,2)]
def RGB_to_hex(RGB):
  ''' [255,255,255] -> "#FFFFFF" '''
  # Components need to be integers for hex to make sense
  RGB = [int(x) for x in RGB]
  return "#"+"".join(["0{0:x}".format(v) if v < 16 else
            "{0:x}".format(v) for v in RGB])

def color_dict(gradient):
  ''' Takes in a list of RGB sub-lists and returns dictionary of
    colors in RGB and hex form for use in a graphing function
    defined later on '''
  return {"hex":[RGB_to_hex(RGB) for RGB in gradient],
      "r":[RGB[0] for RGB in gradient],
      "g":[RGB[1] for RGB in gradient],
      "b":[RGB[2] for RGB in gradient]}

def linear_gradient(start_hex, finish_hex="#FFFFFF", n=10):
  ''' returns a gradient list of (n) colors between
    two hex colors. start_hex and finish_hex
    should be the full six-digit color string,
    inlcuding the number sign ("#FFFFFF") '''
  # Starting and ending colors in RGB form
  s = hex_to_RGB(start_hex)
  f = hex_to_RGB(finish_hex)
  # Initilize a list of the output colors with the starting color
  RGB_list = [s]
  # Calcuate a color at each evenly spaced value of t from 1 to n
  for t in range(1, n):
    # Interpolate RGB vector for color at the current value of t
    curr_vector = [
      int(s[j] + (float(t)/(n-1))*(f[j]-s[j]))
      for j in range(3)
    ]
    # Add it to our list of output colors
    RGB_list.append(curr_vector)

  return color_dict(RGB_list)

def polylinear_gradient(colors, n):
  ''' returns a list of colors forming linear gradients between
      all sequential pairs of colors. "n" specifies the total
      number of desired output colors '''
  # The number of colors per individual linear gradient
  n_out = int(float(n) / (len(colors) - 1))
  # returns dictionary defined by color_dict()
  gradient_dict = linear_gradient(colors[0], colors[1], n_out)

  if len(colors) > 1:
    for col in range(1, len(colors) - 1):
      next = linear_gradient(colors[col], colors[col+1], n_out)
      for k in ("hex", "r", "g", "b"):
        # Exclude first point to avoid duplicates
        gradient_dict[k] += next[k][1:]

  return gradient_dict

In [ ]:
# Define color gradient
color_gradient = polylinear_gradient(['#0000FF','#000000', '#FF0000'], 11)["hex"]

In [ ]:
bins = pd.cut(DE["log2FC"],10,labels=False)

### Subtask 3.5: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.4

In [ ]:
def draw_kegg_map_gradient(map_id,color_list, color_dict):
    """ 
        Render a local PDF of a KEGG map with the passed map ID.
        The pathways are colored according to the log2FC of the first contained gene
        as color gradient. 
    """
    # Get KGML file from KEGG
    pathway = KGML_parser.read(kegg_get(map_id, "kgml"))
    # Iterate over elements and color them
    for element in pathway.genes:
        for name in get_gene_name(element.name):
            if name in diff_reg_genes.index:
                for graphic in element.graphics: # only one element in element.graphics
                    try:
                        graphic.bgcolor = color_list[color_dict[name]]
                    except:
                        continue
                break # break after first hit
            else:
                print 0
    canvas = KGMLCanvas(pathway, import_imagemap=True)
    img_filename = "DayY_InOutput/%s_grad.pdf" % map_id
    canvas.draw(img_filename)
    return PDF(img_filename)

In [ ]:
draw_kegg_map_gradient("mmu04640",color_gradient,bins)